In [2]:
#### DATA TYPES ####

In [66]:
# @dataclasses.dataclass(stuff)
class foo:
    # bar: List[int] = []
    baz: int = []

# a = foo() # bar = []
# a.bar.append(1) # bar [1]
# b = foo() # bar [1]
# print(b.bar)

In [68]:
foo.__annotations__

{'baz': int}

In [70]:
foo.baz

3

In [ ]:
globals()

In [93]:
locals() is globals()

False

In [94]:
type(locals())

ipython_autoimport._AutoImporterMap

In [95]:
type(globals())

dict

In [89]:
def new(a: 1=[]):
    a.append(1)
    print(a)

In [90]:
new.__annotations__

{'a': 1}

In [79]:
name: str

In [80]:
__builtins__.__annotations__

{'name': str, 'foo': int}

In [78]:
globals()['__annotations__']

KeyError: '__annotations__'

In [72]:
new() # prints [1]

[1]


In [73]:
new() # prints [1, 1]

[1, 1]


In [ ]:
def make_new_class(cls):
    class new:
        def __init__(attr=cls.attr for attr in cls.__annotations__)
    return new

In [ ]:
foo()

In [109]:
None.__repr__()

'None'

In [110]:
from typing import List, Dict, Optional, Tuple

@dataclasses.dataclass
class CookingInstruction:
    index: int
    task: str
    duration: int
    attention: bool
    dependencies: Tuple[int] = ()

@dataclasses.dataclass
class ChefTask:
    task_index: int
    when_started: Optional[int]=None
    when_dismissed: Optional[int]=None

@dataclasses.dataclass
class Session:
    tasks: Tuple[CookingInstruction]
    chef_tasks: Dict[str, Tuple[ChefTask, ...]]

"""
1. alice sees put eggs to boil
2. alice presses done!
3. alice sees chop carrots, and a 5 minute timer for eggs
4. alice presses done! (it took her 3 minutes, as expected, she's on time, she's the best.)
5. put carrots in oven, close door, doesn't have to be on.
6. alice sees just timers
7. alice hears egg timer goes off and sees take out eggs
8. alice dismisses timer and goes to sleep. fuck this recipe.
"""

"\n1. alice sees put eggs to boil\n2. alice presses done!\n3. alice sees chop carrots, and a 5 minute timer for eggs\n4. alice presses done! (it took her 3 minutes, as expected, she's on time, she's the best.)\n5. put carrots in oven, close door, doesn't have to be on.\n6. alice sees just timers\n7. alice hears egg timer goes off and sees take out eggs\n8. alice dismisses timer and goes to sleep. fuck this recipe.\n"

In [21]:
def next_unstarted_instruction(session, chef) -> ??:
    """Return the index in the chef's tasks of the first unstarted instruction if exists, otherwise None"""
    # single line!
    raise NotImplementedError

def advance_instruction_for_chef(session, chef, now):
    i = next_unstarted_instruction(session, chef)
    if i is None:
        return session
        
    chef_tasks = {session.chef_tasks[chef2] for chef2 in sessions.chef_tasks if chef2 != chef}
    chef_tasks[chef] = (
        session.chefs[chef][0], session.chefs[chef][1][:i]
        # fix this - what should be here??
        + [ChefTask(
                session.chefs[chef][1][i], now
        )]
        # also this is wrong:
        + session.chefs[chef][1][i + 1:]
    )
    return Session(session.tasks, chef_tasks)

def new_chef_joined(session, chef, now):
    raise NotImplementedError

def chef_leave(session, chef):
    raise NotImplementedError

def dismiss_timer(session, chef, now):
    raise NotImplementedError

def active_timers(session, chef) -> Tuple[ChefTask]:
    raise NotImplementedError

# and more

In [2]:
# Chefs
alice = Chef("Alice")
bob = Chef("Bob")

# Instructions 
i1 = CookingInstruction(0, "Chop onions", 60, True, [])
i2 = CookingInstruction(1, "Boil water", 120, False, [0])
i3 = CookingInstruction(2, "Saute onions", 90, True, [0,1])

# Assignment: index -> list of chef names
assignments = {
    0: ["Alice"],
    1: ["Bob"],
    2: ["Alice", "Bob"]  # both chefs for this instruction
}

# Session
session = Session(
    past=[i1],
    present=[i2],
    future=[i3],
    chefs=[alice, bob],
    assignments=assignments
)

In [3]:
session

Session(past=[CookingInstruction(index=0, task='Chop onions', duration=60, attention=True, dependencies=[])], present=[CookingInstruction(index=1, task='Boil water', duration=120, attention=False, dependencies=[0])], future=[CookingInstruction(index=2, task='Saute onions', duration=90, attention=True, dependencies=[0, 1])], chefs=[Chef(name='Alice'), Chef(name='Bob')], assignments={0: ['Alice'], 1: ['Bob'], 2: ['Alice', 'Bob']})

In [4]:
#### COMPUTATIONS ####


In [9]:
def advance_instruction_for_chef(session, chef_name: str):
    """
    Returns a new session where the given chef's current instruction(s) is(are) marked as past,
    and their next future instruction (if all dependencies are met) moves to present.
    """
    # Move present instruction for chef to past
    new_past = session.past[:]
    new_present = []
    done_indices = {instr.index for instr in new_past}
    
    moved_index = None
    for instr in session.present:
        chefs_for_instr = session.assignments.get(instr.index, [])
        if chef_name in chefs_for_instr and moved_index is None:
            new_past.append(instr)
            moved_index = instr.index
        else:
            new_present.append(instr)
    done_indices = done_indices | ({moved_index} if moved_index is not None else set())

    # Move the next future instruction for chef (if dependencies are met) to present
    new_future = session.future[:]
    added_instr = None
    for i, instr in enumerate(new_future):
        if chef_name in session.assignments.get(instr.index, []):
            # Check dependencies are all in done_indices
            if all(dep in done_indices for dep in instr.dependencies):
                added_instr = instr
                del new_future[i]
                break
    if added_instr:
        new_present.append(added_instr)
    
    return Session(new_past, new_present, new_future, session.chefs[:], dict(session.assignments))

In [10]:
def advance_instruction_for_chef(session, chef_name: str):
    """ 
    Returns a new session where the given chef's current instruction(s) is(are) marked as past,
    and their next future instruction (if all dependencies are met) moves to present.
    """
    # Move present instruction for chef to past
    new_past = session.past[:]
    new_present = []
    done_indices = {instr.index for instr in new_past}
    
    # Move chef's instruction from present to past; keep others in present
    moved_index = None
    for instr in session.present:
        chefs_for_instr = session.assignments.get(instr.index, [])
        if chef_name in chefs_for_instr and moved_index is None:
            new_past.append(instr)
            moved_index = instr.index
        else:
            new_present.append(instr)
    done_indices = done_indices | ({moved_index} if moved_index is not None else set())

    # Move the next future instruction for chef (if dependencies are met) to present
    new_future = session.future[:]
    added_instr = None
    for i, instr in enumerate(new_future):
        if chef_name in session.assignments.get(instr.index, []):
            # Check dependencies are all in done_indices
            if all(dep in done_indices for dep in instr.dependencies):
                added_instr = instr
                del new_future[i]
                break
    if added_instr:
        new_present.append(added_instr)
    
    return Session(new_past, new_present, new_future, session.chefs[:], dict(session.assignments))

In [11]:
session1 = advance_instruction_for_chef(session,'Alice')

In [14]:
session2 = advance_instruction_for_chef(session1,'Bob')

In [15]:
for instr in session2.past:
    print(instr.attention)

True
False


In [17]:
session2

Session(past=[CookingInstruction(index=0, task='Chop onions', duration=60, attention=True, dependencies=[]), CookingInstruction(index=1, task='Boil water', duration=120, attention=False, dependencies=[0])], present=[CookingInstruction(index=2, task='Saute onions', duration=90, attention=True, dependencies=[0, 1])], future=[], chefs=[Chef(name='Alice'), Chef(name='Bob')], assignments={0: ['Alice'], 1: ['Bob'], 2: ['Alice', 'Bob']})

In [18]:
def chef_joins(session, new_chef_name, compile_cooking_instructions):
    """
    Returns a new session where new_chef_name joins.
    Only the future instructions (and their assignments) are recompiled
    using the new chef list and the existing future instructions.
 
    NOTE: The function does not assign a present instruction to the new chef. For this, one should apply 
    the 'advance_instruction_for_chef' function with the new_chef.
    """
    # Step 1: Build the new chef list (avoid duplicate chef)
    chef_names = [c.name for c in session.chefs]
    if new_chef_name in chef_names:
        # Chef already present: return session unchanged
        return Session(
            past=session.past[:],
            present=session.present[:],
            future=session.future[:],
            chefs=session.chefs[:],
            assignments=dict(session.assignments)
        )

    new_chef = Chef(new_chef_name)
    new_chefs = session.chefs[:] + [new_chef]

    # Step 2: Compile new future instructions and assignments using the black box
    # Signature: compile_cooking_instructions(future_instructions, chef_list) -> (instructions, assignments)
    new_future, new_assignments = compile_cooking_instructions(session.future[:], new_chefs)

    # Step 3: Combine old assignments (for past & present) with new future assignments
    preserved_assignments = {idx: val for idx, val in session.assignments.items()
                             if any(idx == instr.index for instr in session.past + session.present)}
    merged_assignments = {**preserved_assignments, **new_assignments}

    # Step 4: Return the new session
    return Session(
        past=session.past[:],
        present=session.present[:],
        future=new_future,
        chefs=new_chefs,
        assignments=merged_assignments
    )
    

In [19]:
### COMPILATION RELATED COMPUTATIONS ###

In [20]:
import tqdm
from subprocess import check_call
#!/usr/bin/env python
# coding: utf-8
# test2
from collections import namedtuple
from pycryptosat import Solver
from itertools import product

cooking_time_unit = 30    # in seconds
assert 60 % unit == 0

Assignment = namedtuple("Assignment", ["attention", "time"])

ModuleNotFoundError: No module named 'pycryptosat'

In [210]:
def preprocess_for_sat(future_instructions, chefs):
    """
    Returns (chef_names, vertices, edges, assignment_map) for SAT encoding.
    Only includes edges between instructions in future_instructions.
    """
    # 1. Chefs
    chef_names = [chef.name for chef in chefs]

    # 2. Vertices (indices of all future instructions)
    vertices = [instr.index for instr in future_instructions]
    vertex_set = set(vertices)  # for fast lookup

    # 3. Edges: only include dependencies where both source and target are in future_instructions
    edges = []
    time_ub = 0
    for instr in future_instructions:
#        print(instr)
        if instr.attention:
            time_ub += instr.duration
        
        for dep in instr.dependencies:
            if dep in vertex_set:
                edges.append((dep, instr.index))

    # 4. Assignment map: only for vertices in the future
    assignment_map = {
        instr.index: Assignment(instr.attention, instr.duration)
        for instr in future_instructions
    }

    return chef_names, vertices, edges, assignment_map, time_ub

In [204]:
session2.past

[CookingInstruction(index=0, task='Chop onions', duration=60, attention=True, dependencies=[]),
 CookingInstruction(index=1, task='Boil water', duration=120, attention=False, dependencies=[0])]

In [212]:
future_instructions = session2.past
chefs =  session.chefs
#print(chefs)
proc, verts, edges, a, timeub = preprocess_for_sat(future_instructions,chefs)


[0, 1]

In [148]:
# Trivial Resorce considerations (for now)
def Resources(v):
    return set([]) #return {"oven"} if "bake" in v else set([])


In [206]:
def recipe2sat(chefs, vertices, edges,a, time_ub, time_unit=cooking_time_unit):
    
    time_slots = range(int(time_ub)*60 // time_unit + 0)
    print('timeslot[0] ==', time_slots[0])
    
    tuples = []
    for p in chefs:
        for v in vertices:
            #for i in range(len(a[v])):
                for t in time_slots:
                    tuples.append((p, t, v))

    tuple2idx = {tpl: idx for idx, tpl in enumerate(tuples, start=1)}
    #idx2tuple = {idx: tpl for tpl, idx in tuple2idx.items()}

    clauses = []

    print("1/6 process does single high attention task")
    for v, u in tqdm.tqdm(list(product(vertices, vertices))):
        #for i, j in product(range(len(a[v])), range(len(a[u]))):
            #if (u != v or i != j) and a[v][i].Attention and a[u][j].Attention:
            #print(u,v,a[v].attention,a[u].attention)
            if (u != v) and a[v].attention and a[u].attention:
                for t in time_slots:
                    for s in range(t, min(t + a[v].time, time_slots[-1] + 1)):
                        for p in Proc:
                            clauses.append(
                                [-tuple2idx[(p, t, v)], -tuple2idx[(p, s, u)]]
                            )


    print("2/6 resource can do one thing at a time")
    for (v, rv), (u, ru) in tqdm.tqdm(
        list(product([(v, rv) for v in Vertices if (rv := Resources(v))], repeat=2))
    ):
        if v != u and ru & rv:
            for p1, p2, t in product(
                chefs, chefs, time_slots
            ):
                for s in range(t, min(t + a[v].time, time_slots[-1] + 1)):
                    clauses.append([-tuple2idx[p1, t, v], -tuple2idx[p2, s, u]])
                                              

    print("3/6 every vertex is done at least once")
    for v in tqdm.tqdm(vertices):
        #for i in range(len(a[v])):
            cls = []
            for p in chefs:
                for t in time_slots:
                    cls.append(tuple2idx[(p, t, v)])
            clauses.append(cls)


    print("4/6 every task is done at most once")
    for p, q in tqdm.tqdm(list(product(chefs, chefs))):
        for t, s in product(time_slots, time_slots):
            if p != q or t != s:
                for v in vertices:
                    #for i in range(len(a[v])):
                        clauses.append([-tuple2idx[(p, t, v)], -tuple2idx[(q, s, v)]])

    #print("5/6 the task are executed serially")
    #for v in tqdm.tqdm(Vertices):
    #    for i in range(len(a[v]) - 1):
    #        for p, q in product(Proc, Proc):
    #            for t, s in product(Times, Times):
    #                if s < t + a[v][i].time:
    #                    clauses.append(
    #                        [-tuple2idx[(p, t, v, i)], -tuple2idx[(q, s, v, i + 1)]]
    #                    )


    print("6/6 the task execution follows the graph structure")
    for v, u in tqdm.tqdm(edges):
        for t, s in product(time_slots, time_slots):
           # if s < t + a[v][len(a[v]) - 1].time:
             if s < t + a[v].time:
                for p, q in product(chefs, chefs):
                    clauses.append(
                        [-tuple2idx[(p, t, v)], -tuple2idx[(q, s, u)]]
                    )
    return clauses, tuples, tuple2idx


In [213]:
cls,tups, tups2inx = recipe2sat(proc, verts, edges, a, timeub)

timeslot[0] == 0
1/6 process does single high attention task


100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 16178.61it/s]


2/6 resource can do one thing at a time


0it [00:00, ?it/s]


3/6 every vertex is done at least once


100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 15391.94it/s]


4/6 every task is done at most once


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 54.33it/s]


6/6 the task execution follows the graph structure


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 98.76it/s]


In [193]:

def sat2IDX(clauses, tuple2idx):
    '''Setup a SAT cryptosolver to solve the SAT cooking clauses, If a solution is found, returns the resulting cooking instructions as a list, if a solution is not found, return False'''
    s = Solver()
    for cls in tqdm.tqdm(clauses):
        s.add_clause(cls)
    sat, solution = s.solve()
    
    if sat:
        idx2tuple = {idx: tpl for tpl, idx in tuple2idx.items()}
        IDX = [idx2tuple[i] for i, s in enumerate(solution) if s]
        return IDX
        #print(            "\n".join(f"{p} {t / (60 // unit):.1f} '{v}'.{i}" for p, t, v, i in sorted(IDX)))
    else:
        return False
    
    
def binarysearch(f,lb,ub):
    """the function search for an minimal input x between lb and ub for which f returns the value True"""
    while lb < ub:
        mid = (lb + ub) // 2
        if f(mid):
            ub = mid
        else: 
            lb = mid + 1
    return lb


def run_with_timeout(f, args, timeout, default=None):
    ctx = multiprocessing.get_context('fork')
    q = ctx.Queue()
    def ff(f, args, q):
        ret = f(*args)
        q.put(ret)
        
    p = ctx.Process(target=ff, args=(f, args, q))
    p.start()
    try:
        ret = q.get(timeout=timeout)
    except queue.Empty:
        p.kill()
        return default
    p.join()
    return ret


In [194]:
sat2IDX(cls,tups2inx)

100%|██████████████████████████████| 165002/165002 [00:00<00:00, 2929564.29it/s]


[('Alice', 1, 0), ('Alice', 61, 1)]

In [ ]:
def IDX2session(IDX):
    

In [39]:
##TODO (for compilation)
#DONE 0 check the status of the recources parts of the SAT. Are we planning to use it at the moment?
#DONE 1 check the SAT can have any number of processors (not just two)
#DONE 2 add time UB estimates in the correct unit form to the preprocessing
#3 See that the compilation of future instructure works (check for type compatibility)
#4 Plan and write a short secion which addresses todo gaps such as working with recources, etc.


['Alice', 'Bob']

[('Alice', 0, 0),
 ('Alice', 1, 0),
 ('Alice', 2, 0),
 ('Alice', 3, 0),
 ('Alice', 4, 0),
 ('Alice', 5, 0),
 ('Alice', 6, 0),
 ('Alice', 7, 0),
 ('Alice', 8, 0),
 ('Alice', 9, 0),
 ('Alice', 10, 0),
 ('Alice', 11, 0),
 ('Alice', 12, 0),
 ('Alice', 13, 0),
 ('Alice', 14, 0),
 ('Alice', 15, 0),
 ('Alice', 16, 0),
 ('Alice', 17, 0),
 ('Alice', 18, 0),
 ('Alice', 19, 0),
 ('Alice', 20, 0),
 ('Alice', 21, 0),
 ('Alice', 22, 0),
 ('Alice', 23, 0),
 ('Alice', 24, 0),
 ('Alice', 25, 0),
 ('Alice', 26, 0),
 ('Alice', 27, 0),
 ('Alice', 28, 0),
 ('Alice', 29, 0),
 ('Alice', 30, 0),
 ('Alice', 31, 0),
 ('Alice', 32, 0),
 ('Alice', 33, 0),
 ('Alice', 34, 0),
 ('Alice', 35, 0),
 ('Alice', 36, 0),
 ('Alice', 37, 0),
 ('Alice', 38, 0),
 ('Alice', 39, 0),
 ('Alice', 40, 0),
 ('Alice', 41, 0),
 ('Alice', 42, 0),
 ('Alice', 43, 0),
 ('Alice', 44, 0),
 ('Alice', 45, 0),
 ('Alice', 46, 0),
 ('Alice', 47, 0),
 ('Alice', 48, 0),
 ('Alice', 49, 0),
 ('Alice', 50, 0),
 ('Alice', 51, 0),
 ('Alice', 52, 0),
 ('